In [1]:
def generate_star():
    train = []
    test = []
    for i in range(1, 5):
        for j in range(1, 5):
            if i==j:
                test.append((i, 5, 5+j))
            else:
                train.append((i, 5, 5+j))
    return train, test

train, test = generate_star()

def add_new_automata(train, test):
    new_train = train.copy()
    new_test = test.copy()
    for i in range(9, 72, 9):
        for sample in train:
            a, b, c = sample
            new_train.append((a+i, b+i, c+i))
        for sample in test:
            a, b, c = sample
            new_test.append((a+i, b+i, c+i))
    return new_train, new_test

train, test = add_new_automata(train, test)

In [2]:
train

[(1, 5, 7),
 (1, 5, 8),
 (1, 5, 9),
 (2, 5, 6),
 (2, 5, 8),
 (2, 5, 9),
 (3, 5, 6),
 (3, 5, 7),
 (3, 5, 9),
 (4, 5, 6),
 (4, 5, 7),
 (4, 5, 8),
 (10, 14, 16),
 (10, 14, 17),
 (10, 14, 18),
 (11, 14, 15),
 (11, 14, 17),
 (11, 14, 18),
 (12, 14, 15),
 (12, 14, 16),
 (12, 14, 18),
 (13, 14, 15),
 (13, 14, 16),
 (13, 14, 17),
 (19, 23, 25),
 (19, 23, 26),
 (19, 23, 27),
 (20, 23, 24),
 (20, 23, 26),
 (20, 23, 27),
 (21, 23, 24),
 (21, 23, 25),
 (21, 23, 27),
 (22, 23, 24),
 (22, 23, 25),
 (22, 23, 26),
 (28, 32, 34),
 (28, 32, 35),
 (28, 32, 36),
 (29, 32, 33),
 (29, 32, 35),
 (29, 32, 36),
 (30, 32, 33),
 (30, 32, 34),
 (30, 32, 36),
 (31, 32, 33),
 (31, 32, 34),
 (31, 32, 35),
 (37, 41, 43),
 (37, 41, 44),
 (37, 41, 45),
 (38, 41, 42),
 (38, 41, 44),
 (38, 41, 45),
 (39, 41, 42),
 (39, 41, 43),
 (39, 41, 45),
 (40, 41, 42),
 (40, 41, 43),
 (40, 41, 44),
 (46, 50, 52),
 (46, 50, 53),
 (46, 50, 54),
 (47, 50, 51),
 (47, 50, 53),
 (47, 50, 54),
 (48, 50, 51),
 (48, 50, 52),
 (48, 50, 54),
 

In [3]:
def get_paths(train):
    train_paths = []
    for i in train:
        train_paths.append(i)
        tup1 = (i[0], i[1])
        train_paths.append(tup1)
        tup2 = (i[1], i[2])
        train_paths.append(tup2)
    return train_paths

train_paths = get_paths(train)
print(len(train_paths))
train_paths = list(sorted(set(get_paths(train))))

288


In [53]:
test_paths = get_paths(test)

In [59]:
test

[(1, 5, 6),
 (2, 5, 7),
 (3, 5, 8),
 (4, 5, 9),
 (10, 14, 15),
 (11, 14, 16),
 (12, 14, 17),
 (13, 14, 18),
 (19, 23, 24),
 (20, 23, 25),
 (21, 23, 26),
 (22, 23, 27),
 (28, 32, 33),
 (29, 32, 34),
 (30, 32, 35),
 (31, 32, 36),
 (37, 41, 42),
 (38, 41, 43),
 (39, 41, 44),
 (40, 41, 45),
 (46, 50, 51),
 (47, 50, 52),
 (48, 50, 53),
 (49, 50, 54),
 (55, 59, 60),
 (56, 59, 61),
 (57, 59, 62),
 (58, 59, 63),
 (64, 68, 69),
 (65, 68, 70),
 (66, 68, 71),
 (67, 68, 72)]

In [82]:
# nejdříve vygenerovat 3 možné cesty pro každou hvězdu z trainu
# (1,5,6) - (1,5), (5,6)
# 1-5 nesmí být distractor
# samplovat dvojice a trojice (50/50) z jiných hvězd jako distractory
# když bude chybět, dát random distractor z jiných hvězd
# potom permutace
# poslední pozice = cílový automat (5 nebo 6)

import itertools
import random
from collections import defaultdict

automata_to_exluded_automata = {i: range(1 + 9*i, 6 + 9*i) for i in range(8)}
star_to_excluded_automata = {i: range(1 + 9*i, 10 + 9*i) for i in range(8)}

def get_distractors(paths, n, train=False):
    combinations = defaultdict(list)
    all_elements = set(element for path in paths for element in path)
    for i, path in enumerate(paths):
        if train:
            group = i // 20
        else:
            group = i // 4

        excluded = set(automata_to_exluded_automata[group])
        candidates = [c for c in paths if not set(c).intersection(excluded)]
        current_star_automata = set(star_to_excluded_automata[group])
        while len(combinations[path]) < n and candidates:
            candidate = random.choice(candidates)
            sample_size = min(len(candidate), random.randint(2, 3))
            inner_list = random.sample(candidate, sample_size)
            
            while len(path) + len(inner_list) < 7:
                available = all_elements - set(path) - set(inner_list) - excluded - current_star_automata
                inner_list.append(random.choice(list(available)))
            
            combinations[path].append(inner_list)
            # candidates.remove(candidate)
            # excluded.update(candidate)

    return combinations

In [85]:
combinations_train = get_distractors(train, 100, True)
combinations_test = get_distractors(test, 5, False)


In [86]:
combinations_test

defaultdict(list,
            {(1, 5, 6): [[27, 22, 23, 63],
              [50, 49, 54, 16],
              [68, 67, 15, 53],
              [32, 34, 18, 56],
              [68, 72, 67, 59]],
             (2, 5, 7): [[26, 23, 21, 67],
              [39, 44, 49, 31],
              [68, 71, 16, 39],
              [18, 13, 26, 68],
              [15, 10, 14, 33]],
             (3, 5, 8): [[15, 14, 44, 48],
              [67, 72, 68, 14],
              [52, 47, 45, 61],
              [32, 36, 31, 14],
              [32, 29, 16, 36]],
             (4, 5, 9): [[14, 11, 43, 58],
              [51, 50, 11, 64],
              [12, 17, 14, 20],
              [61, 56, 59, 27],
              [31, 36, 59, 26]],
             (10, 14, 15): [[40, 41, 45, 63],
              [5, 1, 6, 25],
              [59, 55, 60, 20],
              [52, 50, 47, 68],
              [39, 41, 44, 48]],
             (11, 14, 16): [[34, 32, 21, 62],
              [47, 52, 60, 23],
              [49, 50, 54, 29],
            

In [87]:
# napárovat train a test podle ID
# {id: (train), (test)}

# nejdříve vygenerovat 3 možné cesty pro každou hvězdu z trainu
# (1,5,6) - (1,5), (5,6)
# 1-5 nesmí být distractor
# samplovat dvojice a trojice (50/50) z jiných hvězd jako distractory
# když bude chybět, dát random distractor z jiných hvězd
# potom permutace
# poslední pozice = cílový automat (5 nebo 6)

In [88]:
import random
import itertools

def generate_train(combinations):
    l_of_d = []
    for path, inner_lists in combinations.items():
        for lst in inner_lists:
            # Generate up to 20 permutations of the original list
            perms = list(itertools.permutations(lst))
            selected_perms = random.sample(perms, min(10, len(perms)))
            
            for perm in selected_perms:
                new_lst = list(perm)
                new_lst.append(path[0])
                if len(path) == 2:
                    random.shuffle(new_lst)
                    new_lst.append(path[1])
                    d = {"input": new_lst, "orig_idx": new_lst.index(path[0]), "target_idx": new_lst.index(path[0]) + 73}
                elif len(path) == 3:
                    new_lst.append(path[1])
                    random.shuffle(new_lst)
                    new_lst.append(path[2])
                    d = {"input": new_lst, "orig_idx": new_lst.index(path[1]),"target_idx": new_lst.index(path[0]) + 73}
                l_of_d.append(d)

    return l_of_d

In [89]:
def generate_test(combinations):
    l_of_d = []
    for path, inner_lists in combinations.items():
        for lst in inner_lists:
            new_lst = list(lst)
            new_lst.append(path[0])
            if len(path) == 2:
                random.shuffle(new_lst)
                new_lst.append(path[1])
                d = {"input": new_lst, "orig_idx": new_lst.index(path[0]), "target_idx": new_lst.index(path[0]) + 73}
            elif len(path) == 3:
                new_lst.append(path[1])
                random.shuffle(new_lst)
                new_lst.append(path[2])
                d = {"input": new_lst, "orig_idx": new_lst.index(path[1]),"target_idx": new_lst.index(path[0]) + 73}
            l_of_d.append(d)

    return l_of_d

In [93]:
result_test = generate_test(combinations_test)
result_train = generate_train(combinations_train)

In [94]:
# udělat permutace a data pro test, přidat index 

In [95]:
data = {"train": result_train, "test": result_test}
import pickle
with open ("new_data.pkl", "wb") as f:
    pickle.dump(data, f)

In [97]:
len(data["train"])

96000